In [53]:
import numpy as np, pandas as pd

import re
import nltk
from nltk.corpus import stopwords
import unicodedata
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize

from time import time
from pandas import DataFrame
from pathlib import Path

from nltk.sentiment import SentimentIntensityAnalyzer

In [54]:
# This is a dataset for classification that has been generated by some experts as part of the dynabench project: https://dynabench.org/tasks/hs and https://arxiv.org/abs/2012.15761
data_gen=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/Dynamically-Generated-Hate-Speech-Dataset-main/Dynamically Generated Hate Dataset v0.2.3.csv",index_col=0)
data_gen.head()


,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [55]:
#data_gen = data_gen.drop(columns=['acl.id', 'X1','round.base','annotator','round','acl.id.matched'], axis=1)
#data_gen_2_combine = data_gen.drop(columns=[ 'type','target','level','split'], axis=1)
#data_gen_2_combine.columns = ['text', 'label']
#data_gen_2_combine.label.value_counts()

In [56]:
#re labelling 1 as hate speech to combine 
#data_gen_2_combine = data_gen_2_combine.replace({'label': {'hate': 1, 'nothate': 0}})
#data_gen_2_combine.label.value_counts()

In [57]:
# this a dataset that has been collated from three other sources from: https://towardsdatascience.com/how-to-create-your-own-hate-tweet-detector-704508c34cd0
#data_comb=pd.read_csv("Data/combined.csv", index_col=0)
#data_comb.head()

In [58]:
# combining the compiled dataset and the one from towardsdatascience
#combined = pd.concat([data_comb, data_gen_2_combine], ignore_index=1)
#combined.label.value_counts()
#combined.to_csv('/Users/rebeccaharrison/Documents/Python/Bootcamp/Data/combined_final.csv') 

In [59]:
# preaparing dataset for analysis with different types of hate speech
data_gen['target'].value_counts()
data_gen["target"] = data_gen["target"].fillna("")

# Creating variables that define target based on religion
data_gen['Religion']=0
data_gen.loc[data_gen['target'].str.contains('jew'), 'Religion'] = 1 # -> Jewish people
data_gen.loc[data_gen['target'].str.contains('mus'), 'Religion'] = 1 # -> Muslims
data_gen.loc[data_gen['target'].str.contains('muswom'), 'Religion'] = 1 # -> Jewish people
data_gen.loc[data_gen['target'].str.contains('other.religion'), 'Religion'] = 1 # -> other religion
data_gen['Jewish people']=0 # -> Jewish people
data_gen.loc[data_gen['target'].str.contains('jew'), 'Jewish people'] = 1 # -> Jewish people
data_gen['Muslims']=0 # -> Muslims
data_gen.loc[data_gen['target'].str.contains('mus'), 'Muslims'] = 1 # -> Muslims
data_gen['Muslim women']=0 # -> Muslim women
data_gen.loc[data_gen['target'].str.contains('muswom'), 'Muslim women'] = 1 # -> Muslim women

#creating a class based variable (removed due to lack of data)
#data_gen['Class']=0 # -> Working class people
#data_gen.loc[data_gen['target'].str.contains('working'), 'Class'] = 1

# Creating an ageist classification
data_gen['Oldist']=0 # -> Elderly people
data_gen.loc[data_gen['target'].str.contains('old'), 'Oldist'] = 1

# Support of hate groups variable
data_gen['Support']=0
data_gen.loc[data_gen['target'].str.contains('nazi'), 'Support'] = 1
data_gen.loc[data_gen['target'].str.contains('hitler'), 'Support'] = 1

# Creating an immigration status based variable
data_gen['Immigration status']=0
data_gen.loc[data_gen['target'].str.contains('immig'), 'Immigration status'] = 1 # -> Immigrants
data_gen.loc[data_gen['target'].str.contains('asylum'), 'Immigration status'] = 1  # -> Asylum seekers
data_gen.loc[data_gen['target'].str.contains('ref'), 'Immigration status'] = 1 # -> Refugees
data_gen.loc[data_gen['target'].str.contains('for'), 'Immigration status'] = 1  # -> Foreigners

# creating an ableness target based variable
data_gen['Ableness / disability']=0
data_gen.loc[data_gen['target'].str.contains('dis'), 'Ableness / disability'] = 1  # -> People with disabilities

# Creating variables that define target based on gender
data_gen['Gender']=0
data_gen.loc[data_gen['target'].str.contains('muswom'), 'Gender'] = 1 # -> Muslim women
data_gen.loc[data_gen['target'].str.contains('indigwom'), 'Gender'] = 1 # -> Indigenous Women
data_gen.loc[data_gen['target'].str.contains('wom'), 'Gender'] = 1 # -> Women
data_gen.loc[data_gen['target'].str.contains('blawom'), 'Gender'] = 1 # -> Black women
data_gen.loc[data_gen['target'].str.contains('blaman'), 'Gender'] = 1 # -> Black men
data_gen.loc[data_gen['target'].str.contains('trans'), 'Gender'] = 1 # -> Trans people
data_gen.loc[data_gen['target'].str.contains('gendermin'), 'Gender'] = 1 # -> Gender minorities,
data_gen.loc[data_gen['target'].str.contains('asiwom'), 'Gender'] = 1 # -> Asian women

data_gen['Indigenous Women']=0 # -> Indigenous Women
data_gen.loc[data_gen['target'].str.contains('indigwom'), 'Indigenous Women'] = 1 # -> Indigenous Women
data_gen['Women']=0 # -> Women 
data_gen.loc[data_gen['target'].str.contains('wom'), 'Women'] = 1 # -> Women
data_gen['Black women']=0 # -> Black women
data_gen.loc[data_gen['target'].str.contains('blawom'), 'Black women'] = 1 # -> Black women
data_gen['Black men']=0 # -> Black men
data_gen.loc[data_gen['target'].str.contains('blaman'), 'Black men'] = 1 # -> Black men
data_gen['Trans people']=0 # -> Trans people
data_gen.loc[data_gen['target'].str.contains('trans'), 'Trans people'] = 1 # -> Trans people
data_gen['Gender minorities']=0 # -> Gender minorities, 
data_gen.loc[data_gen['target'].str.contains('gendermin'), 'Gender minorities'] = 1 # -> Gender minorities,
data_gen['Asian women']=0 # -> Asian women
data_gen.loc[data_gen['target'].str.contains('asiwom'), 'Asian women'] = 1 # -> Asian women

# Creating variables based on sexuality and sexual preference
data_gen['Sexuality and Sexual preference']=0
data_gen.loc[data_gen['target'].str.contains('bis'), 'Sexuality and Sexual preference'] = 1 # -> Bisexual
data_gen.loc[data_gen['target'].str.contains('gay'), 'Sexuality and Sexual preference'] = 1 # -> Gay people (both men and women)
data_gen.loc[data_gen['target'].str.contains('gayman'), 'Sexuality and Sexual preference'] = 1 # -> Gay men
data_gen.loc[data_gen['target'].str.contains('gaywom'), 'Sexuality and Sexual preference'] = 1 # -> Lesbians 
data_gen.loc[data_gen['target'].str.contains('lgbtq'), 'Sexuality and Sexual preference'] = 1 # -> lgbtq

data_gen['Bisexual']=0 # -> Bisexual
data_gen.loc[data_gen['target'].str.contains('bis'), 'Bisexual'] = 1 # -> Bisexual
data_gen['Gay']=0 # -> Gay people (both men and women)
data_gen.loc[data_gen['target'].str.contains('gay'), 'Gay'] = 1 # -> Gay people (both men and women)
data_gen['Gay men']=0 # -> Gay men
data_gen.loc[data_gen['target'].str.contains('gayman'), 'Gay men'] = 1 # -> Gay men
data_gen['Gay women']=0 # -> Lesbians    
data_gen.loc[data_gen['target'].str.contains('gaywom'), 'Gay women'] = 1 # -> Lesbians 



# Creating an ethnicity based variable
data_gen['Ethnicity/Race']=0
data_gen.loc[data_gen['target'].str.contains('mixed'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('ethnic.minority'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('indig'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('non.white'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('trav'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('african'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('asi'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('east'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('south'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('chinese'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('pak'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('arab'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('eastern european'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('russian'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('pol'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('hispanic'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('bla'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('other.national'), 'Ethnicity/Race'] = 1
data_gen.loc[data_gen['target'].str.contains('immig'), 'Ethnicity/Race'] = 1 # -> Immigrants
data_gen.loc[data_gen['target'].str.contains('asylum'), 'Ethnicity/Race'] = 1  # -> Asylum seekers
data_gen.loc[data_gen['target'].str.contains('ref'), 'Ethnicity/Race'] = 1 # -> Refugees
data_gen.loc[data_gen['target'].str.contains('for'), 'Ethnicity/Race'] = 1  # -> Foreigners

# Creating a black race based variable
data_gen['Black people']=0
data_gen.loc[data_gen['target'].str.contains('bla'), 'Black people'] = 1
data_gen.loc[data_gen['target'].str.contains('african'), 'Black people'] = 1

# Creating other race or ethnicity based individual variables
data_gen['Asian']=0 # -> Asians
data_gen.loc[data_gen['target'].str.contains('asi'), 'Asian'] = 1
data_gen['East asian']=0 # -> East Asians
data_gen.loc[data_gen['target'].str.contains('east'), 'East asian'] = 1
data_gen['South Asians']=0 # -> South Asians (e.g. Indians)
data_gen.loc[data_gen['target'].str.contains('south'), 'South Asian'] = 1
data_gen['Chinese']=0 # -> Chinese people
data_gen.loc[data_gen['target'].str.contains('chinese'), 'Chinese'] = 1
data_gen['Pakistani']=0 # -> Pakistanis
data_gen.loc[data_gen['target'].str.contains('pak'), 'Pakistani'] = 1
data_gen['Arab']=0 # -> Arabs, including people from the Middle East
data_gen.loc[data_gen['target'].str.contains('arab'), 'Arab'] = 1
data_gen['Eastern European']=0 # -> Eastern Europeans
data_gen.loc[data_gen['target'].str.contains('eastern european'), 'Eastern European'] = 1
data_gen['Russian']=0 # -> Russian people
data_gen.loc[data_gen['target'].str.contains('russian'), 'Russian'] = 1
data_gen['Polish']=0 # -> Polish people
data_gen.loc[data_gen['target'].str.contains('pol'), 'Polish'] = 1
data_gen['Hispanic']=0 # -> Hispanic people, including latinx and Mexicans
data_gen.loc[data_gen['target'].str.contains('hispanic'), 'Hispanic'] = 1

data_gen['Mixed race']=0 #-> Mixed race background
data_gen.loc[data_gen['target'].str.contains('mixed'), 'Mixed race'] = 1
data_gen['Ethnic minority']=0 # -> Ethnic Minorities
data_gen.loc[data_gen['target'].str.contains('ethnic minority'), 'Ethnic minority'] = 1
data_gen['Indigenous people']=0 # -> Indigenous people
data_gen.loc[data_gen['target'].str.contains('indig'), 'Indigenous people'] = 1

data_gen['Non-white']=0 # -> Non-whites (attacked as 'non-whites', rather than specific non-white groups which are generally addressed separately)
data_gen.loc[data_gen['target'].str.contains('non-white'), 'Non-white'] = 1
data_gen['Traveller']=0 # -> Travellers (including Roma, gypsies)
data_gen.loc[data_gen['target'].str.contains('trav'), 'Traveller'] = 1

data_gen['Working class']=0 # -> Travellers (including Roma, gypsies)
data_gen.loc[data_gen['target'].str.contains('wc'), 'Working class'] = 1


data_gen.head()

,acl.id,X1,text,label,type,target,level,split,round.base,annotator,...,Eastern European,Russian,Polish,Hispanic,Mixed race,Ethnic minority,Indigenous people,Non-white,Traveller,Working class
1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,0
2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,0
3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,...,0,0,0,0,0,0,0,0,0,0
4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,...,0,0,0,0,0,0,0,0,0,0
5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# removing stopwords from text

# Get a list of stopwords in English
stop_words = stopwords.words('english')

# Remove stopwords from the 'text' column in the dataframe
data_gen['text_nostop'] = data_gen['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
data_gen.head()

,acl.id,X1,text,label,type,target,level,split,round.base,annotator,...,Russian,Polish,Hispanic,Mixed race,Ethnic minority,Indigenous people,Non-white,Traveller,Working class,text_nostop
1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,dalits lowlives
2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,gay people burden society
3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,...,0,0,0,0,0,0,0,0,0,Arabs welcome
4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,...,0,0,0,0,0,0,0,0,0,I'm saying actually eliminate heebs.. I wish n...
5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,...,0,0,0,0,0,0,0,0,0,bananas black people


In [61]:
data_gen = data_gen.replace({'label': {'hate': 1, 'nothate': 0}})
data_gen = data_gen.rename(columns={'label':'Hate speech'})



In [62]:
# these are the hate words
search_words = pd.read_csv('/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/Dynamically-Generated-Hate-Speech-Dataset-main/hurtlex-master/lexica/EN/1.2/hurtlex_EN.tsv', sep='\t')

categories=search_words['category'].unique()
categories
search_words.head()

# count the number of times words from the hurtlex appear

# by category
for cat in categories:
    word_list="(" + "|".join(list(search_words['lemma'].loc[search_words['category']==cat])) + ")"
    data_gen[cat] = data_gen['text_nostop'].str.count(word_list)

# overall
words="(" + "|".join(list(search_words['lemma'])) + ")"
data_gen['hurtlex'] = data_gen['text_nostop'].str.count(words)
data_gen['hurtlex'].head()





1    1
2    2
3    0
4    0
5    2
Name: hurtlex, dtype: int64

In [63]:
data_gen

,acl.id,X1,text,Hate speech,type,target,level,split,round.base,annotator,...,asm,pr,dmc,asf,ddp,ps,pa,rci,ddf,hurtlex
1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,1,0,0,0,0,0,0,1
2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,2
3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,...,0,0,0,0,0,0,0,0,0,0
4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,...,0,0,0,0,0,0,0,0,0,0
5,acl10000,5,bananas are for black people,1,notgiven,notgiven,original,test,1,QiOKkCi7F8,...,0,0,1,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117100,acl9974,41228,Indian culture is so backwards,1,notgiven,notgiven,original,test,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,0,0
118100,acl11574,1752,"Women don't want equality, they just want to b...",1,derogation,wom,original,train,2,CAgNlUizNm,...,0,0,0,0,0,0,0,0,0,0
119100,acl13446,3832,fuck covid,0,none,none,original,train,2,CAgNlUizNm,...,1,0,0,0,0,0,0,0,0,1
1205,acl13457,3844,This computer is garbage,0,none,none,original,train,2,eLGzdD8Tvb,...,0,0,0,0,0,0,0,0,0,1


In [64]:
# count capital letters
data_gen['capital_letters'] = data_gen['text_nostop'].str.findall(r'[A-Z]').str.len()
data_gen['capital_letters']
data_gen.head()

,acl.id,X1,text,Hate speech,type,target,level,split,round.base,annotator,...,pr,dmc,asf,ddp,ps,pa,rci,ddf,hurtlex,capital_letters
1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,1,0,0,0,0,0,0,1,0
2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,0,0,0,0,2,0
3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,...,0,0,0,0,0,0,0,0,0,1
4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,...,0,0,0,0,0,0,0,0,0,2
5,acl10000,5,bananas are for black people,1,notgiven,notgiven,original,test,1,QiOKkCi7F8,...,0,1,0,0,1,0,0,0,2,0


In [65]:
sia=SentimentIntensityAnalyzer()

In [66]:
# run sentiment analyser from NLTK
data_gen['polarity'] = data_gen['text_nostop'].apply(lambda x: sia.polarity_scores(str(x)))
# Change data structure so its not nested
data_gen = pd.concat(
    [data_gen.drop([
       'polarity'], axis=1), 
     data_gen['polarity'].apply(pd.Series)], axis=1)
data_gen.head()

,acl.id,X1,text,Hate speech,type,target,level,split,round.base,annotator,...,ps,pa,rci,ddf,hurtlex,capital_letters,neg,neu,pos,compound
1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,1,0,0.756,0.244,0.000,-0.4767
2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,...,0,0,0,0,2,0,0.492,0.508,0.000,-0.4404
3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,...,0,0,0,0,0,1,0.000,0.250,0.750,0.4588
4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,...,0,0,0,0,0,2,0.000,0.748,0.252,0.4019
5,acl10000,5,bananas are for black people,1,notgiven,notgiven,original,test,1,QiOKkCi7F8,...,1,0,0,0,2,0,0.000,1.000,0.000,0.0000


In [67]:
data_gen=data_gen.sort_values('X1')
data_gen=data_gen.reset_index()
data_gen=data_gen.drop(['index','acl.id','X1','round.base','annotator','round','acl.id.matched'],axis=1)

data_gen.to_csv('prepared_hate_data_train.csv')

In [68]:
data_gen

,text,Hate speech,type,target,level,split,Religion,Jewish people,Muslims,Muslim women,...,ps,pa,rci,ddf,hurtlex,capital_letters,neg,neu,pos,compound
0,dalits are lowlives,1,notgiven,notgiven,original,train,0,0,0,0,...,0,0,0,0,1,0,0.756,0.244,0.000,-0.4767
1,gay people are a burden to our society,1,notgiven,notgiven,original,train,0,0,0,0,...,0,0,0,0,2,0,0.492,0.508,0.000,-0.4404
2,Arabs are not welcome,1,notgiven,notgiven,original,train,0,0,0,0,...,0,0,0,0,0,1,0.000,0.250,0.750,0.4588
3,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,0,0,0,0,...,0,0,0,0,0,2,0.000,0.748,0.252,0.4019
4,bananas are for black people,1,notgiven,notgiven,original,test,0,0,0,0,...,1,0,0,0,2,0,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41139,I hate drags lol,0,none,none,original,test,0,0,0,0,...,0,0,0,0,1,1,0.659,0.000,0.341,-0.3818
41140,My washing machine is bitch,0,none,none,original,test,0,0,0,0,...,0,0,0,0,2,1,0.559,0.441,0.000,-0.5859
41141,Fulham is just a bunch of losers,0,none,none,original,test,0,0,0,0,...,0,0,0,0,1,1,0.630,0.370,0.000,-0.5267
41142,K I L L the T R A N S,1,notgiven,notgiven,original,test,0,0,0,0,...,0,0,0,0,0,9,0.000,0.000,0.000,0.0000


In [69]:
# importing and preparing the twitter and reddit data sets for analysis
twitter=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/twitter_clean.csv")
reddit=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/reddit_clean.csv")

/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_1983/1548800313.py:2: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  twitter=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/twitter_clean.csv")


In [70]:
reddit

,Unnamed: 0,subreddit,created,text,language
0,0,worldcup,1670707019.0,"Exciting! France advances play Croatia, realit...",en
1,1,worldcup,1670706978.0,Bellingham good nothing special. His nationali...,en
2,2,worldcup,1670706906.0,"Congratulations Budweiser Man Match, France’s ...",en
3,3,worldcup,1670706846.0,Does google already know final game? Or typo? ...,en
4,4,worldcup,1670706802.0,We getting rematch 2018 something different?nan,en
...,...,...,...,...,...
39288,39288,NaN,1670214814.0,Japan vs Korea would amazing,en
39289,39289,NaN,1670235006.0,This Belgium Germany got knocked,en
39290,39290,NaN,1670214868.0,Yes! It would extremely fun I’d happy whoever ...,en
39291,39291,NaN,1670198597.0,"At international level, FIFA four official lan...",en


In [71]:
# removing stopwords from text

# Remove stopwords from the 'text' column in the dataframe
twitter['text_nostop'] = twitter['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

twitter.head()





,Unnamed: 0,id,created,text,possibly_sensitive,user,verified,rule,language,text_nostop
0,5,1.601650e+18,2022-12-10T18:29:58.000Z,WE LOVE LIAM PAYNE 💗\n #EnglandVsFrance #Engla...,False,Judit,False,England France,en,WE LOVE LIAM PAYNE 💗 #EnglandVsFrance #England...
1,6,1.601650e+18,2022-12-10T18:29:57.000Z,What has Islam got to do with it?,False,Antonio Plescia,False,Worldcup,en,What Islam got it?
2,11,1.601650e+18,2022-12-10T18:29:57.000Z,2002. That was also the Cup where Nigeria bo...,False,Shiv Ramdas Traing To Rite Buk,True,England France,en,2002. That also Cup Nigeria bounced Argentina....
3,13,1.601650e+18,2022-12-10T18:29:58.000Z,Grant Wahl was a healthy individual just like...,False,Porfirio Diaz,False,Worldcup,en,Grant Wahl healthy individual like many thousa...
4,18,1.601650e+18,2022-12-10T18:29:58.000Z,#ssfootball France 🇫🇷 2 England 🇬🇧 1,False,Vinny Munda,False,England France,en,#ssfootball France 🇫🇷 2 England 🇬🇧 1


In [72]:
# Remove stopwords from the 'text' column in the dataframe
reddit=reddit.dropna(subset=['text'])
reddit['text_nostop'] = reddit['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
reddit=reddit.dropna(subset=['text_nostop'])
reddit.head()

/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_1983/148079104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit['text_nostop'] = reddit['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


,Unnamed: 0,subreddit,created,text,language,text_nostop
0,0,worldcup,1670707019.0,"Exciting! France advances play Croatia, realit...",en,"Exciting! France advances play Croatia, realit..."
1,1,worldcup,1670706978.0,Bellingham good nothing special. His nationali...,en,Bellingham good nothing special. His nationali...
2,2,worldcup,1670706906.0,"Congratulations Budweiser Man Match, France’s ...",en,"Congratulations Budweiser Man Match, France’s ..."
3,3,worldcup,1670706846.0,Does google already know final game? Or typo? ...,en,Does google already know final game? Or typo? ...
4,4,worldcup,1670706802.0,We getting rematch 2018 something different?nan,en,We getting rematch 2018 something different?nan


In [73]:
stopwords

<WordListCorpusReader in '/Users/rebeccaharrison/nltk_data/corpora/stopwords'>

In [74]:

categories=search_words['category'].unique()
categories
search_words.head()

# count the number of times words from the hurtlex appear

# by category
for cat in categories:
    word_list="(" + "|".join(list(search_words['lemma'].loc[search_words['category']==cat])) + ")"
    twitter[cat] = twitter['text'].str.count(word_list)

# overall
words="(" + "|".join(list(search_words['lemma'])) + ")"
twitter['hurtlex'] = twitter['text'].str.count(words)
twitter['hurtlex'].head()


0    0
1    2
2    1
3    3
4    2
Name: hurtlex, dtype: int64

In [75]:
# count capital letters
twitter['capital_letters'] = twitter['text_nostop'].str.findall(r'[A-Z]').str.len()
twitter['capital_letters']
twitter.head()


reddit['capital_letters'] = reddit['text_nostop'].str.findall(r'[A-Z]').str.len()
reddit['capital_letters']
reddit.head()

,Unnamed: 0,subreddit,created,text,language,text_nostop,capital_letters
0,0,worldcup,1670707019.0,"Exciting! France advances play Croatia, realit...",en,"Exciting! France advances play Croatia, realit...",5
1,1,worldcup,1670706978.0,Bellingham good nothing special. His nationali...,en,Bellingham good nothing special. His nationali...,4
2,2,worldcup,1670706906.0,"Congratulations Budweiser Man Match, France’s ...",en,"Congratulations Budweiser Man Match, France’s ...",7
3,3,worldcup,1670706846.0,Does google already know final game? Or typo? ...,en,Does google already know final game? Or typo? ...,2
4,4,worldcup,1670706802.0,We getting rematch 2018 something different?nan,en,We getting rematch 2018 something different?nan,1


In [76]:
categories=search_words['category'].unique()
categories
search_words.head()

# count the number of times words from the hurtlex appear

# by category
for cat in categories:
    word_list= words="(" + "|".join(list(search_words['lemma'].loc[search_words['category']==cat])) + ")"
    reddit[cat] = reddit['text'].str.count(word_list)

# overall
words="(" + "|".join(list(search_words['lemma'])) + ")"
reddit['hurtlex'] = reddit['text'].str.count(words)
reddit['hurtlex'].head()



0    2
1    1
2    2
3    3
4    1
Name: hurtlex, dtype: int64

In [77]:


# run sentiment analyser from NLTK
twitter['polarity'] = twitter['text_nostop'].apply(lambda x: sia.polarity_scores(str(x)))
# Change data structure so its not nested
twitter = pd.concat(
    [twitter.drop([
       'polarity'], axis=1), 
     twitter['polarity'].apply(pd.Series)], axis=1)
twitter.head()
reddit['polarity'] = reddit['text_nostop'].apply(lambda x: sia.polarity_scores(str(x)))
# Change data structure
reddit = pd.concat(
    [reddit.drop([ 'polarity'], axis=1), 
     reddit['polarity'].apply(pd.Series)], axis=1)
reddit
reddit.head()

,Unnamed: 0,subreddit,created,text,language,text_nostop,capital_letters,qas,cds,is,...,ddp,ps,pa,rci,ddf,hurtlex,neg,neu,pos,compound
0,0,worldcup,1670707019.0,"Exciting! France advances play Croatia, realit...",en,"Exciting! France advances play Croatia, realit...",5,0,1,0,...,0,0,0,0,0,2,0.000,0.590,0.410,0.7393
1,1,worldcup,1670706978.0,Bellingham good nothing special. His nationali...,en,Bellingham good nothing special. His nationali...,4,0,1,0,...,0,0,0,0,0,1,0.132,0.586,0.281,0.3699
2,2,worldcup,1670706906.0,"Congratulations Budweiser Man Match, France’s ...",en,"Congratulations Budweiser Man Match, France’s ...",7,0,2,0,...,1,0,0,0,0,2,0.000,0.589,0.411,0.6360
3,3,worldcup,1670706846.0,Does google already know final game? Or typo? ...,en,Does google already know final game? Or typo? ...,2,0,1,0,...,0,0,0,0,1,3,0.000,1.000,0.000,0.0000
4,4,worldcup,1670706802.0,We getting rematch 2018 something different?nan,en,We getting rematch 2018 something different?nan,1,0,0,0,...,0,0,0,0,0,1,0.000,1.000,0.000,0.0000


In [87]:
twitter=twitter.dropna(subset=['text_nostop'])
reddit=reddit.dropna(subset=['text_nostop'])

In [88]:
twitter=twitter.reset_index()
reddit=reddit.reset_index()
twitter=twitter.drop(['index'],axis=1)
reddit=reddit.drop(['index'],axis=1)

In [89]:
reddit.to_csv('reddit_prepared.csv')
twitter.to_csv('twitter_prepared.csv')